In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

## Understand the Dataset

In [2]:
trials_df = spark.read.json('dataset.json', multiLine=True) # read json file

In [3]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [4]:
trials_df.printSchema() # view schema

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [5]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [6]:
columns = ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant'
 ] 

In [7]:
trials_df.select(columns).show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [8]:
from pyspark.sql import functions as fn

In [9]:
# counting null values per column

trials_df.select([ fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



## Cleaning

In [10]:
# flaten df
# address null values
# rename columns

In [11]:
# flaten df

new_trials_df = trials_df.select(columns)
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [12]:
# rename columns
new_column_names = {
    'ageofparticipant': 'age_of_participant'
    , 'branch': 'clinic_branch'
    , 'name': 'head_clinician'
    , 'role': 'assistants_role'
    , 'experimentenddate': 'experiment_end_date'
    , 'experimentstartdate': 'experiment_start_date'
    , 'noofhourspassedatfirstreaction': 'hours_passed_at_first_reaction'
    , 'sideeffectsonparticipant': 'observed_side_effect'
}
new_trials_df = new_trials_df.withColumnsRenamed(new_column_names)
new_trials_df.show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|
+------------------+-------------+--------------+---------------+---------+-----

In [13]:
new_trials_df.describe().show() 

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+--------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+--------------------+
|  count|              3586|         3586|          3586|           3477|     3586|                3586|                 3586|                          3513|         3533|                3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|         null|                null|
| stddev|2.3066401927555233|       

In [14]:
# fill columns with null values

new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistants_role': 'unknown', 'hours_passed_at_first_reaction': 0})

In [15]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+--------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+-------------+--------------------+
|  count|              3586|         3586|          3586|           3586|     3586|                3586|                 3586|                          3586|         3586|                3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|              43.9771332961517|         null|                null|
| stddev|2.3066401927555233|       

In [16]:
new_trials_df.show(5)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|
|                17|      Ontario|          Saul|          nurse|  Placebo|     

## Data Transformation

In [17]:
new_trials_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistants_role: string (nullable = false)
 |-- drug_used: string (nullable = true)
 |-- experiment_end_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- hours_passed_at_first_reaction: long (nullable = false)
 |-- conclusion: string (nullable = false)
 |-- observed_side_effect: string (nullable = true)



#### Tansforming the experiment_end_date and experiment_start_date

In [18]:
from pyspark.sql import types as dtypes
from pyspark.sql.functions import col, from_unixtime
from pyspark.sql.types import TimestampType

In [19]:
new_trials_df = new_trials_df\
    .withColumn('start_ts', fn.from_unixtime(fn.col('experiment_start_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn('start_ts', fn.col('start_ts').cast(dtypes.TimestampType()))\
            .withColumn('end_ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
            .withColumn('end_ts', fn.col('end_ts').cast(dtypes.TimestampType()))

In [20]:

# Show the DataFrame with the converted timestamp columns
new_trials_df.show()

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-------------------+-------------------+
|age_of_participant|clinic_branch|head_clinician|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|           start_ts|             end_ts|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-------------------+-------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|2021-04-01 02:00:00|2021-05-01 02:00:00|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1

In [21]:
'''# Convert 'experiment_start_date' to a timestamp column
new_trials_df = new_trials_df.withColumn(
    'start_ts',
    from_unixtime(col('experiment_start_date') / 1000, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
)

# Convert 'experiment_end_date' to a timestamp column
new_trials_df = new_trials_df.withColumn(
    'end_ts',
    from_unixtime(col('experiment_end_date') / 1000, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())
)

# Show the DataFrame with the converted timestamp columns
new_trials_df.show()'''

"# Convert 'experiment_start_date' to a timestamp column\nnew_trials_df = new_trials_df.withColumn(\n    'start_ts',\n    from_unixtime(col('experiment_start_date') / 1000, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())\n)\n\n# Convert 'experiment_end_date' to a timestamp column\nnew_trials_df = new_trials_df.withColumn(\n    'end_ts',\n    from_unixtime(col('experiment_end_date') / 1000, 'yyyy-MM-dd HH:mm:ss').cast(TimestampType())\n)\n\n# Show the DataFrame with the converted timestamp columns\nnew_trials_df.show()"